In [1]:
import openai
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
from datetime import datetime
import time

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [4]:
df = pd.read_csv(r"wikidata_sample_df.csv")

In [5]:
df = df.drop(columns=['Unnamed: 0', 'class.type', 'item.type', 'itemLabel.xml:lang', 'itemLabel.type', 'classLabel.type','subclass.type', 'subclass.value', 'subclassLabel.xml:lang', 'subclassLabel.type', 'subclassLabel.value'])

In [6]:
cats = ['human','album','village','film','river', 'business','musical group','literary work','mountain','television series']

In [7]:
df = df[df['classLabel.value'].isin(cats)]

In [8]:
len(df)

46463

In [9]:
df['classLabel.value'].value_counts()

human                35005
album                 2514
village               2072
film                  1872
river                 1195
business              1104
musical group          913
literary work          825
mountain               482
television series      481
Name: classLabel.value, dtype: int64

In [10]:
def get_correct_article(text):
    ## simple logic to make articles match their targets
    if text[0].lower() in ['a', 'e', 'i', 'o', 'u']:
        article_variant = "an"
    else:
        article_variant = "a"
    
    return article_variant

# def setSubsetQuestion(subset, superset):
  
#     article_variant = get_correct_article(superset)

#     ## should we inject quotes around the itemLabel.value 
#     query_basic = f"True or false: {df.loc[index, 'itemLabel.value']} is {article_variant} {df.loc[index, 'classLabel.value']}"
#     df.loc[index, 'set_subset_q'] = query_basic
#     print(query_basic)
#     response_basic = openai.Completion.create(model="text-davinci-003", prompt=query_basic, temperature=0, max_tokens=1000)
#     df.loc[index, 'set_subset_a'] = response_basic['choices'][0]['text']
#     print(response_basic['choices'][0]['text'])

#     return

def ask_question(subset, superset):
    time.sleep(1)
    article_variant = get_correct_article(superset)
    set_subset = f"{subset} is {article_variant} {superset}?"
    boilerplate = f"Answer True or False, then explain your rationale, and provide a numeric confidence score as a percentage out of 100: {set_subset}"
    response_basic = openai.Completion.create(model="text-davinci-003", prompt=boilerplate, temperature=0, max_tokens=1000)
    
    return response_basic['choices'][0]['text']

In [38]:
ask_question('Barney', 'dinosaur')

'\n\nFalse. Barney is a character from the children\'s television show "Barney and Friends". Rationale: Barney is a fictional character, not a real dinosaur. Confidence Score: 100%.'

In [39]:
ask_question('Britney Spears', 'singer')

'\n\nTrue. Britney Spears is an American singer, songwriter, dancer, and actress. She is one of the most successful and influential pop artists of all time. Confidence score: 100%.'

In [25]:
ask_question('Rick Grimes', 'human')

'\n\nTrue. Rick Grimes is a fictional character from the television series The Walking Dead. He is a human and not a zombie. Confidence score: 100%.'

In [20]:
df = df.sort_values(by='classLabel.value').reset_index(drop=True)

In [54]:
n = 50
list_df = [unfinished_df[i:i+n] for i in range(0,len(unfinished_df),n)]

In [55]:
len(list_df)

476

In [ ]:
starttime = datetime.now()
print(f"Started at {starttime.strftime('%m/%d/%Y, %H:%M:%S')}")

for sub_df in tqdm(list_df):
#     df['set subset response'] = df.progress_apply(lambda x: ask_question(x['itemLabel.value'], x['classLabel.value']), axis=1)
    sub_df['set subset response'] = sub_df.apply(lambda x: ask_question(x['itemLabel.value'], x['classLabel.value']), axis=1)
#     sub_df['classLabel.value'].apply(dummy_func)
    
stoptime = datetime.now()
print(f"Finished at {stoptime.strftime('%m/%d/%Y, %H:%M:%S')}")
print(f"Ran in {str(stoptime - starttime)}")

Started at 03/30/2023, 19:56:07


  0%|                                                                                                                                                                            | 0/476 [00:00<?, ?it/s]/tmp/ipykernel_2186893/1296158883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['set subset response'] = sub_df.apply(lambda x: ask_question(x['itemLabel.value'], x['classLabel.value']), axis=1)
  0%|▎                                                                                                                                                                | 1/476 [01:37<12:53:07, 97.66s/it]/tmp/ipykernel_2186893/1296158883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [34]:
list_df[0]

,item.value,class.value,itemLabel.value,classLabel.xml:lang,classLabel.value,set subset response
0,http://www.wikidata.org/entity/Q9615903,http://www.wikidata.org/entity/Q482994,Anime Alive 2008,en,album,\n\nFalse. Anime Alive 2008 is an anime conven...
1,http://www.wikidata.org/entity/Q1452745,http://www.wikidata.org/entity/Q482994,The Tide and Its Takers,en,album,\n\nFalse. The Tide and Its Takers is a song b...
2,http://www.wikidata.org/entity/Q4839372,http://www.wikidata.org/entity/Q482994,Back to Avalon,en,album,\n\nFalse. Back to Avalon is a song by Kenny L...
3,http://www.wikidata.org/entity/Q3987034,http://www.wikidata.org/entity/Q482994,The Final Cuts,en,album,\n\nFalse. The Final Cuts is a compilation alb...
4,http://www.wikidata.org/entity/Q3437907,http://www.wikidata.org/entity/Q482994,Rock a Memphis,en,album,\n\nFalse. Rock a Memphis is not an album. I a...
5,http://www.wikidata.org/entity/Q1172070,http://www.wikidata.org/entity/Q482994,Every Six Seconds,en,album,\n\nFalse. Every Six Seconds is a song by the ...
6,http://www.wikidata.org/entity/Q4838372,http://www.wikidata.org/entity/Q482994,Baby Monkey,en,album,\n\nFalse. Baby Monkey is a song by the band G...
7,http://www.wikidata.org/entity/Q3077256,http://www.wikidata.org/entity/Q482994,Forged in Fire,en,album,\n\nFalse. Forged in Fire is a 1983 heavy meta...
8,http://www.wikidata.org/entity/Q6684477,http://www.wikidata.org/entity/Q482994,Lost in the Ozone,en,album,\n\nFalse. Lost in the Ozone is a song by Comm...
9,http://www.wikidata.org/entity/Q2904404,http://www.wikidata.org/entity/Q482994,Bird 'n' Roll,en,album,\n\nFalse. Bird 'n' Roll is a song by the band...


In [36]:
bigdf = pd.concat(list_df)

In [48]:
run2df = pd.concat(list_df)

In [49]:
unfinished_df = run2df[~run2df['set subset response'].notnull()]

In [50]:
len(unfinished_df)

23763

In [52]:
len(run1df[run1df['set subset response'].notnull()])

13350

In [51]:
len(run2df[run2df['set subset response'].notnull()])

9350

In [53]:
23763 + 13350 + 9350

46463

In [46]:
run1df = bigdf.copy()

In [47]:
run1df

,item.value,class.value,itemLabel.value,classLabel.xml:lang,classLabel.value,set subset response
0,http://www.wikidata.org/entity/Q9615903,http://www.wikidata.org/entity/Q482994,Anime Alive 2008,en,album,\n\nFalse. Anime Alive 2008 is an anime conven...
1,http://www.wikidata.org/entity/Q1452745,http://www.wikidata.org/entity/Q482994,The Tide and Its Takers,en,album,\n\nFalse. The Tide and Its Takers is a song b...
2,http://www.wikidata.org/entity/Q4839372,http://www.wikidata.org/entity/Q482994,Back to Avalon,en,album,\n\nFalse. Back to Avalon is a song by Kenny L...
3,http://www.wikidata.org/entity/Q3987034,http://www.wikidata.org/entity/Q482994,The Final Cuts,en,album,\n\nFalse. The Final Cuts is a compilation alb...
4,http://www.wikidata.org/entity/Q3437907,http://www.wikidata.org/entity/Q482994,Rock a Memphis,en,album,\n\nFalse. Rock a Memphis is not an album. I a...
...,...,...,...,...,...,...
46458,http://www.wikidata.org/entity/Q664339,http://www.wikidata.org/entity/Q532,North Robinson,en,village,NaN
46459,http://www.wikidata.org/entity/Q5727065,http://www.wikidata.org/entity/Q532,Dowlatabad,en,village,NaN
46460,http://www.wikidata.org/entity/Q4857790,http://www.wikidata.org/entity/Q532,"Baqerabad, Hamadan",en,village,NaN
46461,http://www.wikidata.org/entity/Q7979386,http://www.wikidata.org/entity/Q532,"Week, Devon",en,village,NaN


In [ ]:
df.to_csv("set_subset_responses.csv", index=False)

In [77]:
list_df[0]

,item.value,class.value,itemLabel.value,classLabel.xml:lang,classLabel.value
6,http://www.wikidata.org/entity/Q88847,http://www.wikidata.org/entity/Q5,Waldemar Kmentt,en,human
7,http://www.wikidata.org/entity/Q118871,http://www.wikidata.org/entity/Q5,Heinrich Scharrer,en,human
10,http://www.wikidata.org/entity/Q197355,http://www.wikidata.org/entity/Q5,Zhang Zilin,en,human
17,http://www.wikidata.org/entity/Q363478,http://www.wikidata.org/entity/Q5,Pierre Pelot,en,human
18,http://www.wikidata.org/entity/Q375994,http://www.wikidata.org/entity/Q5,Björn Ferry,en,human
...,...,...,...,...,...
1602,http://www.wikidata.org/entity/Q1905521,http://www.wikidata.org/entity/Q5,Martina Seifert,en,human
1603,http://www.wikidata.org/entity/Q1910555,http://www.wikidata.org/entity/Q5,Matthäus Tabbert,en,human
1604,http://www.wikidata.org/entity/Q1933905,http://www.wikidata.org/entity/Q482994,End of Silence,en,album
1610,http://www.wikidata.org/entity/Q2062697,http://www.wikidata.org/entity/Q5,Louise Closser Hale,en,human
